## Steg 1 - Hämta in relevant information i for av csv-filer

For the evaluation will the following data be required: 

• Datum- och tidsstämpel 

• Flöde för värmeöverföringsmediet 

• Inlopps- och utloppstemperatur från solfångarfältet 

• Temperatur vid flödesmätning 

• Vindhastighet 

• Omgivningstemperatur 

• Solinstrålning (pyranometer och pyrheliometer, se Figur 6) 

• Temperaturderivatan – ϑm/dt för varje tidssteg, se (2)1F
4
. Där ϑm är skillnaden i 
medeltemperatur över solfångaren/fältet för det uppmätta tidssteget. 

• Temperatur (och flöde) i varje delkrets för att kunna ta hänsyn till rörförlusterna. 

• Eftersom solfångarna har inbyggd solföljning så behöver också solfångarnas vinkel 
(mot horisontalplanet) mätas och loggas kontinuerligt.

#### Utmaningar:
- alla csv-filer får samma namn när de laddas ned från databas. Hur ska vi hantera det samtidigt som de hålls isär?
- Struktur på Notebooken? Hur vill vi att resultat och output ska se ut?
- Hur mycket ska programmet kunna känna av själv (Ex. vilken typ av storhet som mäts baserat på variabelnamn?), och vad ska "hårdkodas"
- Idag kan endast 4 parametrar/csv-fil hämtas ned. Opraktiskt. FIXAT
- Table of contents. Tänk igenom rubriksättnig
- remember to cite PVlib


In [1]:
# Import packages

import pandas as pd
import numpy as np

# Plotting tools
import plotly.express as px
import matplotlib.pyplot as plt

In [2]:
# Inhämtning av data

In [3]:
'''Function takes name of csv-file. 
    Return: two pandas dataframes'''

def import_file(filename):
    # Här sparar jag ned en annan tillfällig fil. Borde gå att lösa snyggare. 
    # Gör detta för att få bort " - tecken, vilka gör hanteringen av data lite jobbig. 
    # Ramez jobbar på bättre exportering av data här. Jag låter funktionen vara
    
    with open(filename) as f_in, open("temp.csv", 'w') as f_out:
        for line in f_in:
            line = line.replace('"','')
            f_out.write(line)

    import pandas as pd
    df = pd.read_csv('temp.csv',sep = ',',engine = 'python')

    # remove temp. file
    import os
    os.remove('temp.csv')
    
    df2 = df.pivot(index = 'Timestamp', columns = 'Channel Name', values = 'Value')

    return df2

In [4]:
df=import_file('mult_data.csv')
df.head(5)

Channel Name,SVS-GT2A_TEMPERATURE,SVS-GT2B_TEMPERATURE,SVS-GT3A_TEMPERATURE,SVS-GT3B_TEMPERATURE
Timestamp,,,,
2021-09-01 00:00:02,54.7,54.6,46.9,46.5
2021-09-01 00:00:12,54.7,54.6,46.9,46.5
2021-09-01 00:00:22,54.7,54.6,46.9,46.5
2021-09-01 00:00:32,54.7,54.6,46.9,46.5
2021-09-01 00:00:42,54.7,54.6,46.9,46.5


In [5]:
# VAD ÄR Cp & rho??
cp = 3915
rho = 969 # For 30% Propylene, But varies with temp. 

Q =  rho*flow_rate*cp*dt


NameError: name 'flow_rate' is not defined

In [ ]:
fig = px.line(newFrame, x= "Timestamp", y=Q, labels={"y": "Q"})
fig.show()

In [6]:
# (df[df["Channel Name"] == 'SVS_MEAN_DNI']["Value"])

fig = px.line(x=df["Timestamp"].unique(), y=df[df["Channel Name"] == 'SVS_MEAN_DNI']["Value"],
             labels={'x':'Timeline','y':'Mean DNI'})

fig.show()

KeyError: 'Timestamp'

In [7]:
# Attempt to get DHI vs. time plot
import os
import itertools
import matplotlib.pyplot as plt
import pandas as pd
import pvlib

from pvlib.location import Location
from pvlib.iotools import read_tmy3



In [ ]:
from pvlib import clearsky, atmosphere, solarposition
from pvlib.location import Location

hsand = Location(62.632, 17.937, 'Europe/Stockholm')
times = pd.date_range(start='2021-09-01', end='2021-09-09', freq='1h', tz=hsand.tz)
cs = hsand.get_clearsky(times)  # ineichen with climatology table by default

## VALIDATION 

Validation includes: "Data type (ex. integer, float, string)
Range (ex. A number between 35-40)
Uniqueness (ex. Postal code)
Consistent expressions (ex. Using one of St., Str, Street)
No null values"


In [ ]:
import matplotlib.pyplot as plt
cs["dni"].plot(label = "DNI")
cs["ghi"].plot(label = "GHI")
cs["dhi"].plot(label = "DHI")
plt.legend()
plt.ylabel('Irradiance $W/m^2$');
plt.title('Ineichen, climatological turbidity');

In [ ]:
measured_DNI = df[df["Channel Name"] == 'SVS_MEAN_DNI']["Value"]
modeled_DNI = cs["dni"][0:-1]
modeled_GHI = cs["ghi"][0:-1]
fig = px.line(x=df["Timestamp"].unique(), y=[modeled_DNI,measured_DNI],
             labels={"x": "Timestamp", "value": "Irradiance W/m2", "modeled_DNI":"Modelled DNI, sunny whether",
                    "measured_DNI":"Measured Mean", "y[0]": "Model"})

fig.show()

$$
\dot{Q}  = \dot{m}* c_p * \Delta T = \rho *\dot{V} * c_{cp} * \Delta T
$$
$$
\Delta T = T_{in} - T_{out}dot{V}
$$

$m$ = Mass flowrate of fluid

$C_p$ = Specific heat capacity = 12.9 kJkg$^{-1}$K$^{-1}$ (enligt testpapper) ?


the specific heat capacity of water is 4184 J⋅kg−1⋅K−1

Sen: 
- Kika närmare på parametrar
- Hur ser datastream ut? Behöver vi något filter? (low pass, etc?)
- Validera data
- Implementera ekvationer. 
- Hitta en fin, solig dag. Försök återskapa Dashboard data.

### Fit data to QDT equation

The extracted power $\dot{Q}$ is in accordance to the quasi dynamic test method for liquid heating collectors modelled as
$$
\dot{Q}=A_{\mathrm{G}}\left[\begin{array}{l} \eta_{0, \mathrm{~b}} K_{\mathrm{b}}\left(\theta_{\mathrm{L}}, \theta_{\mathrm{T}}\right) G_{\mathrm{b}}+\eta_{0, \mathrm{~b}} K_{\mathrm{d}} G_{\mathrm{d}}-a_{1}\left(\vartheta_{\mathrm{m}}-\vartheta_{\mathrm{a}}\right)-a_{2}\left(\vartheta_{\mathrm{m}}-\vartheta_{a}\right)^{2}-a_{3} u^{\prime}\left(\vartheta_{\mathrm{m}}-\vartheta_{\mathrm{a}}\right)+ \\
a_{4}\left(E_{\mathrm{L}}-\sigma T_{\mathrm{a}}^{4}\right)-a_{5}\left(d \vartheta_{\mathrm{m}} / d t\right)-a_{6} u^{\prime} G-a_{7} u^{\prime}\left(E_{\mathrm{L}}-\sigma T_{\mathrm{a}}^{4}\right)-a_{8}\left(\vartheta_{\mathrm{m}}-\vartheta_{\mathrm{a}}\right)^{4}
\end{array}\right].
$$

Where parameters as given by the Solar Keymark test for induvidual collectors are given by 

|$A_G$|$\eta_{0,b}$|$a_1$|$a_2$|$a_1$|$a_3$|$a_4$|$a_5$|$a_6$|$a_7$|$a_8$|$K_d$|
|---|---|---|---|---|---|---|---|---|---|---|---|
|6.04|0.697|0.73|0.000|0.000|0.000|0.00|0|0.000|0.00|0.00|0.12|

$K_b$, $G_b$, $G_d$, $\vartheta_{\mathrm{m}}, \vartheta_{\mathrm{a}}$ from data. Need to calculate $K_{\mathrm{b}}\left(\theta_{\mathrm{L}}, \theta_{\mathrm{T}}\right)$, behöver här en solpositionsperäkning. Behöver alltså theta, dvs. vinkel mot solfångarutan. 

From given table, simplify the power equation to 


$$
\dot{Q} = A_{\mathrm{G}} \left( \eta_{0, \mathrm{~b}} K_{\mathrm{b}}\left(\theta_{\mathrm{L}}, \theta_{\mathrm{T}}\right) G_{\mathrm{b}}+\eta_{0, \mathrm{~b}} K_{\mathrm{d}} G_{\mathrm{d}}-a_{1}\left(\vartheta_{\mathrm{m}}-\vartheta_{\mathrm{a}}\right)  \right)
$$

In [ ]:
a1, a2, a3, a4, a5,a6,a7,a8, Kd = 0.73, 0.000, 0.000, 0.00, 0, 0.000, 0.00, 0.00, 0.12
Ag,etab = 6.04,0.697

In [ ]:
# Behöver en funktion för att få till Kb(thetaL,thetaT) som funktion av tid? 

In [ ]:
Kb = 1 # BARA HITTEPÅ. ÄNDRA DETTA DELUXE
# Ta in: Temp in, ut, amb, Gd, Gb, angles?? Eller hur får vi infallsvinkel theta?

Q_simpleModel = Ag*(etab*Kb*Gb + etab*Kd*Gd -a1*(Tm-Ta) - a2*(Tm-Ta)**2)


In [ ]:
# Kika lite på temperaturer
filename='Feb28GT.csv'
df = pd.read_csv(filename).pivot(index = 'Timestamp', columns = 'Channel Name', values = 'Value')

df['GT1_mean'] = df[df.columns[:8]].mean(axis=1)
fig = px.line(df, x=df.index, y=df.columns[:8],
              labels={'value': 'Temperature'})
fig.show()

In [ ]:
df_max = pd.DataFrame({'Channel':df[df.columns[:8]].idxmax(axis=1),'Value':df[df.columns[:8]].max(axis=1)})
#df = px.data.gapminder().query("continent=='Oceania'")
df_max.describe()

In [ ]:
fig = px.histogram(df_max, x = 'Channel')
#scatter(df_max, x=df_max.index, y="Value", color='Channel')
fig.show()

In [ ]:
GT1,GT2 = np.array(df['SVS-GT1:1_TEMPERATURE']),np.array(df['SVS-GT1:2_TEMPERATURE'])
GT3,GT4 = np.array(df['SVS-GT1:3_TEMPERATURE']),np.array(df['SVS-GT1:4_TEMPERATURE'])
GT5,GT6 = np.array(df['SVS-GT1:5_TEMPERATURE']),np.array(df['SVS-GT1:6_TEMPERATURE'])

In [ ]:
from scipy.stats import f_oneway

# compare samples
stat, p = f_oneway(GT1,GT2,GT3)
print('Statistics=%.3f, p=%.3f' % (stat, p))
# interpret
alpha = 0.05
if p > alpha:
    print('Same distributions (fail to reject H0)')
else:
    print('Different distributions (reject H0)')

In [ ]:
QDT_IAM = [1.00, 0.99, 0.98, 0.96, 0.90, 0.80, 0.64,
           0.43, 0.21, 0.00]

Angles = [0, 10, 20, 30, 40, 50, 60, 70, 80, 90]

fig, ((ax1, ax2), (ax3, ax4)) = plt.subplots(2, 2, figsize=(10, 8))
fig.suptitle('QDT and polyfit')

degrees = np.arange(1,5)
for (deg, ax) in zip(degrees, fig.get_axes()):
    k = np.polyfit(Angles,QDT_IAM,deg)
    #Evaluating New ydata
    y=np.polyval(k,Angles)

    Y = lambda Angles: np.polyval(k,Angles)
    sum_of_residuals = sum((Y(Angles) - QDT_IAM)**2)
    RSE = np.sqrt(sum_of_residuals/(len(Angles)-2))
    print('Sum of residuals for polynomial degree {0} is {1}'.format(deg, sum_of_residuals))
    print('Residual Standar Error for polynomial degree {0} is {1}'.format(deg, RSE))
    
    ax.scatter(Angles,QDT_IAM,150,label='original data')
    ax.plot(Angles,Y(Angles),'r:',label=' fit of deg ={}'.format(deg))
    
    ax.grid(axis='both',which='major',color=[166/255,166/255,166/255], linestyle='-', linewidth=1)
    #ax.minorticks_on()
    #ax.grid(axis='both',which='minor',color=[166/255,166/255,166/255,0.4], linestyle=':', linewidth=0.5)
    ax.scatter(45,Y(45),100,label='$\theta$ = {}'.format(45))
        
    ax.legend()
    ax.set_ylabel('IAM')
    ax.set_xlabel('Incidence angle')
    


In [ ]:
import statsmodels.api as sm

#define response variable
y = Angles

#define predictor variables
x = [QDT_IAM,Y(Angles)]

#add constant to predictor variables
x = sm.add_constant(x)

#fit linear regression model
#model = sm.OLS(y, x).fit()

X = np.array([float(x) for x in Angles])
X

$$
\text {Residual Rum of Squares (RSS) }=\sum_{i=1}^{n}\left(y_{i}-\hat{y}_{i}\right)^{2}
$$
$$
\text{Residual Standar Error (RSE)} = \sqrt{\frac{RSS}{n-2}}
$$
$(n=\text{# of data points})$
VAD ÄR "BRA" SUM OF RESIDUALS?

In [ ]:
QDT_IAM = [1.00, 0.99, 0.98, 0.96, 0.90, 0.80, 0.64,
           0.43, 0.21, 0.00]
SS_IAM = [1.00, 0.99, 0.99, 0.98, 0.96, 0.91, 0.77, 0.53, 
          0.18, 0.00]
Angles = [0, 10, 20, 30, 40, 50, 60, 70, 80, 90] #zenith angles
IAM_df = pd.DataFrame({#'Angle':Angles,
                       'QDT-method':QDT_IAM,
                      'SS-method':SS_IAM}, index = Angles)

IAM_df.index.names = ['Angle']

IAM_df.T

In [ ]:
from pvlib import solarposition
tz = 'Europe/Stockholm'
lat, lon = 62.6329, 17.938 # degrees

times = pd.date_range('2021-10-13 00:00:00', '2022-10-13 23:58:50', freq='10s', tz=tz)

#times = pd.date_range('2019-01-01 00:00:00', '2020-01-01', closed='left',
 #                     freq='H', tz=tz)

solpos = solarposition.get_solarposition(times, lat, lon)

In [ ]:
solpos

In [ ]:
solpos['elevation'].plot()

In [ ]:
import numpy as np
# curve-fit() function imported from scipy
from scipy.optimize import curve_fit
 
from matplotlib import pyplot as plt
 
# numpy.linspace with the given arguments
# produce an array of 40 numbers between 0
# and 10, both inclusive
x = np.linspace(0, 10, num = 40)
 
 
# y is another array which stores 3.45 times
# the sine of (values in x) * 1.334.
# The random.normal() draws random sample
# from normal (Gaussian) distribution to make
# them scatter across the base line
y = 3.45 * np.sin(1.334 * x) + np.random.normal(size = 40)
 
# Test function with coefficients as parameters
def test(x, a, b):
    return a * np.sin(b * x)
 
# curve_fit() function takes the test-function
# x-data and y-data as argument and returns
# the coefficients a and b in param and
# the estimated covariance of param in param_cov
param, param_cov = curve_fit(test, x, y)
 
print("Sine function coefficients:")
print(param)
print("Covariance of coefficients:")
print(param_cov)
 
# ans stores the new y-data according to
# the coefficients given by curve-fit() function
ans = (param[0]*(np.sin(param[1]*x)))
 
'''Below 4 lines can be un-commented for plotting results
using matplotlib as shown in the first example. '''
 
plt.plot(x, y, 'o', color ='red', label ="data")
plt.plot(x, ans, '--', color ='blue', label ="optimized data")
plt.legend()
plt.show()

In [ ]:
import matplotlib.pyplot as plt
from scipy.optimize import curve_fit
import numpy as np

def func(x, a, b, c):
    return a * np.exp(-b * x) + c

xdata = np.linspace(0, 4, 50)
y = func(xdata, 2.5, 1.3, 0.5)
rng = np.random.default_rng()
y_noise = 0.2 * rng.normal(size=xdata.size)
ydata = y + y_noise
plt.scatter(xdata, ydata, label='data')

popt, pcov = curve_fit(func, xdata, ydata)
plt.plot(xdata, func(xdata, *popt), 'r-',
         label='fit: a=%5.3f, b=%5.3f, c=%5.3f' % tuple(popt))
popt



popt, pcov = curve_fit(func, xdata, ydata, bounds=(0, [3., 1., 0.5]))
plt.plot(xdata, func(xdata, *popt), 'g--',
         label='fit: a=%5.3f, b=%5.3f, c=%5.3f' % tuple(popt))

plt.xlabel('x')
plt.ylabel('y')
plt.legend()
plt.show()


In [ ]:
QDT_IAM = [1.00, 0.99, 0.98, 0.96, 0.90, 0.80, 0.64,
           0.43, 0.21, 0.00]

Angles = [0, 10, 20, 30, 40, 50, 60, 70, 80, 90]

In [ ]:
theta = np.array(Angles)*np.pi/180
K = 1 - (np.tan(theta/2))**np.array(QDT_IAM)
plt.plot(Angles,K,label='ISO eq.')
plt.plot(Angles,IAM_df['QDT-method'],label='QDT-method')
plt.plot(Angles,IAM_df['SS-method'],label = 'SS-method')
plt.legend()
plt.show()
